In [121]:
import argparse
import pandas as pd
import os
import itertools
from sklearn.decomposition import PCA
import numpy as np
from scipy import stats
import math

In [122]:
def avg_max_t(X, meu=0):
    # rows of x are number of subjects and columns keypoints
    # meu represents the population mean assumed under some hypothesis
    # m is the number of samples, x_bar is sample mean, sigma is sample variance
#     print("No. of features: ", X.shape[1])
    t_arr = np.zeros((X.shape[1]))
    p_arr = np.zeros((X.shape[1]))
    for j in range(X.shape[1]):  
        # print(stats.ttest_1samp(X[:, j], popmean=meu))
        # print(X[:, j])
        t_arr[j] = stats.ttest_1samp(X[:, j], popmean=meu).statistic
        p_arr[j] = stats.ttest_1samp(X[:, j], popmean=meu).pvalue
        if math.isnan(t_arr[j]):
            print(t_arr[j])
            t_arr[j] = 0
            p_arr[j] = 1
        # print(t_arr[j])
    # t_arr = np.nan_to_num(t_arr)
    t_arr = np.absolute(t_arr)
    return [np.average(t_arr), np.amax(t_arr)]


In [123]:
def HotellingTest(X):
    statistic = 0
    try:
        n, p = X.shape
        delta = np.mean(X, axis=0).reshape(1, -1)
        Sx = sample_covariance(X)
        S_pooled = Sx/n
        t_squared = delta@np.linalg.inv(S_pooled)@delta.T
        t1 = delta@np.linalg.inv(Sx)@delta.T
        t2 = t1/np.linalg.det(np.linalg.inv(Sx))
        t3 = np.linalg.det(Sx)
        t4 = delta@delta.T
#         statistic = t_squared[0,0]*(n-p)/(p*(n-1))
        statistic = t_squared[0,0]
    except Exception as e:
        print(str(e))

    return statistic

In [124]:
def sample_covariance(X):
    temp = np.zeros((X.shape[1], X.shape[1]))
    n = X.shape[0]
    for i in range(X.shape[1]):
        for j in range(X.shape[1]):
            temp[i, j] = (1/(n-1))*(X[:, i] - np.mean(X[:, i]))@(X[:, j] - np.mean(X[:, j]))
    return temp

In [125]:
def var_explained(X, X_approx):
    # X and X_approx need to be numpy array
    return 100 - 100*(np.linalg.norm(X - X_approx)**2)/(np.linalg.norm(X)**2)

In [126]:

def t_data_creation(X, save_path, var_threshold):
    
    columns = ['Frame_No', 'Avg_t', 'Max_t', 'Avg_t2', 'Max_t2', 't2_with_PCA']

    cols_x = [s for s in X.columns if 'delta_x' in s]
    cols_y = [s for s in X.columns if 'delta_y' in s]
    cols_xy = cols_x+cols_y

    final_data = []

    X_temp = X[cols_xy].copy()

    pca_X_temp = X_temp.to_numpy()

    for n_comp in range(1, pca_X_temp.shape[1]):
        pca = PCA(n_components=n_comp)
        pca.fit(pca_X_temp)

        X_temp_transform = pca_X_temp@pca.components_.T
        ve = var_explained(pca_X_temp, X_temp_transform@pca.components_)

        if ve>=var_threshold:
            print(n_comp, ve)
            break

    for frame_no in sorted(X['Frame_No'].unique()):
        print("Frame no: ", frame_no)
        bivar_HT2 = []
        for col_x, col_y in zip(cols_x, cols_y):
            bivar_HT2.append(HotellingTest(X[X['Frame_No']==frame_no][[col_x]+[col_y]].to_numpy()))
        avg_bivar_HT2 = np.mean(bivar_HT2)
        max_bivar_HT2 = np.amax(bivar_HT2)

        X_temp_fno = X_temp[X['Frame_No']==frame_no].to_numpy()
        temp = avg_max_t(X_temp_fno)
        avg_t, max_t = temp[0], temp[1]

        t2_with_PCA = HotellingTest(X_temp_transform[X['Frame_No']==frame_no])

        res = [frame_no, avg_t, max_t, avg_bivar_HT2, max_bivar_HT2, t2_with_PCA]

        final_data.append(res)

    df = pd.DataFrame(final_data, columns=columns)
    df.to_csv(save_path)
    return df


In [127]:
df_disfa = pd.read_csv("Data/DISFA_KPMs.csv", index_col=0)

In [128]:
#Dropping the registered keypoints in data

drop_keypoints = [0, 16, 27, 33, 39, 42]

drop_columns = []
for kp in drop_keypoints:
    drop_columns.append('delta_x'+str(kp)+'_Normalized')
for kp in drop_keypoints:
    drop_columns.append('delta_y'+str(kp)+'_Normalized')

df_disfa.drop(columns=drop_columns, inplace=True)

In [129]:
df_disfa['Subject_No'].unique()

array([21,  3,  4, 30, 12, 24,  2, 23, 31,  8, 13,  7,  9,  6, 27, 28, 17,
       10, 29,  1,  5, 32, 16, 11, 26, 18, 25])

In [130]:
df_t_data = t_data_creation(df_disfa.copy(), "Data/all_keypoint_metrics.csv", var_threshold=90)

5 90.76643622984686
Frame no:  0
Frame no:  1
Frame no:  2
Frame no:  3
Frame no:  4
Frame no:  5
Frame no:  6
Frame no:  7
Frame no:  8
Frame no:  9
Frame no:  10
Frame no:  11
Frame no:  12
Frame no:  13
Frame no:  14
Frame no:  15
Frame no:  16
Frame no:  17
Frame no:  18
Frame no:  19
Frame no:  20
Frame no:  21
Frame no:  22
Frame no:  23
Frame no:  24
Frame no:  25
Frame no:  26
Frame no:  27
Frame no:  28
Frame no:  29
Frame no:  30
Frame no:  31
Frame no:  32
Frame no:  33
Frame no:  34
Frame no:  35
Frame no:  36
Frame no:  37
Frame no:  38
Frame no:  39
Frame no:  40
Frame no:  41
Frame no:  42
Frame no:  43
Frame no:  44
Frame no:  45
Frame no:  46
Frame no:  47
Frame no:  48
Frame no:  49
Frame no:  50
Frame no:  51
Frame no:  52
Frame no:  53
Frame no:  54
Frame no:  55
Frame no:  56
Frame no:  57
Frame no:  58
Frame no:  59
Frame no:  60
Frame no:  61
Frame no:  62
Frame no:  63
Frame no:  64
Frame no:  65
Frame no:  66
Frame no:  67
Frame no:  68
Frame no:  69
Frame no: 

Frame no:  555
Frame no:  556
Frame no:  557
Frame no:  558
Frame no:  559
Frame no:  560
Frame no:  561
Frame no:  562
Frame no:  563
Frame no:  564
Frame no:  565
Frame no:  566
Frame no:  567
Frame no:  568
Frame no:  569
Frame no:  570
Frame no:  571
Frame no:  572
Frame no:  573
Frame no:  574
Frame no:  575
Frame no:  576
Frame no:  577
Frame no:  578
Frame no:  579
Frame no:  580
Frame no:  581
Frame no:  582
Frame no:  583
Frame no:  584
Frame no:  585
Frame no:  586
Frame no:  587
Frame no:  588
Frame no:  589
Frame no:  590
Frame no:  591
Frame no:  592
Frame no:  593
Frame no:  594
Frame no:  595
Frame no:  596
Frame no:  597
Frame no:  598
Frame no:  599
Frame no:  600
Frame no:  601
Frame no:  602
Frame no:  603
Frame no:  604
Frame no:  605
Frame no:  606
Frame no:  607
Frame no:  608
Frame no:  609
Frame no:  610
Frame no:  611
Frame no:  612
Frame no:  613
Frame no:  614
Frame no:  615
Frame no:  616
Frame no:  617
Frame no:  618
Frame no:  619
Frame no:  620
Frame no: 

Frame no:  1095
Frame no:  1096
Frame no:  1097
Frame no:  1098
Frame no:  1099
Frame no:  1100
Frame no:  1101
Frame no:  1102
Frame no:  1103
Frame no:  1104
Frame no:  1105
Frame no:  1106
Frame no:  1107
Frame no:  1108
Frame no:  1109
Frame no:  1110
Frame no:  1111
Frame no:  1112
Frame no:  1113
Frame no:  1114
Frame no:  1115
Frame no:  1116
Frame no:  1117
Frame no:  1118
Frame no:  1119
Frame no:  1120
Frame no:  1121
Frame no:  1122
Frame no:  1123
Frame no:  1124
Frame no:  1125
Frame no:  1126
Frame no:  1127
Frame no:  1128
Frame no:  1129
Frame no:  1130
Frame no:  1131
Frame no:  1132
Frame no:  1133
Frame no:  1134
Frame no:  1135
Frame no:  1136
Frame no:  1137
Frame no:  1138
Frame no:  1139
Frame no:  1140
Frame no:  1141
Frame no:  1142
Frame no:  1143
Frame no:  1144
Frame no:  1145
Frame no:  1146
Frame no:  1147
Frame no:  1148
Frame no:  1149
Frame no:  1150
Frame no:  1151
Frame no:  1152
Frame no:  1153
Frame no:  1154
Frame no:  1155
Frame no:  1156
Frame no

Frame no:  1608
Frame no:  1609
Frame no:  1610
Frame no:  1611
Frame no:  1612
Frame no:  1613
Frame no:  1614
Frame no:  1615
Frame no:  1616
Frame no:  1617
Frame no:  1618
Frame no:  1619
Frame no:  1620
Frame no:  1621
Frame no:  1622
Frame no:  1623
Frame no:  1624
Frame no:  1625
Frame no:  1626
Frame no:  1627
Frame no:  1628
Frame no:  1629
Frame no:  1630
Frame no:  1631
Frame no:  1632
Frame no:  1633
Frame no:  1634
Frame no:  1635
Frame no:  1636
Frame no:  1637
Frame no:  1638
Frame no:  1639
Frame no:  1640
Frame no:  1641
Frame no:  1642
Frame no:  1643
Frame no:  1644
Frame no:  1645
Frame no:  1646
Frame no:  1647
Frame no:  1648
Frame no:  1649
Frame no:  1650
Frame no:  1651
Frame no:  1652
Frame no:  1653
Frame no:  1654
Frame no:  1655
Frame no:  1656
Frame no:  1657
Frame no:  1658
Frame no:  1659
Frame no:  1660
Frame no:  1661
Frame no:  1662
Frame no:  1663
Frame no:  1664
Frame no:  1665
Frame no:  1666
Frame no:  1667
Frame no:  1668
Frame no:  1669
Frame no

Frame no:  2121
Frame no:  2122
Frame no:  2123
Frame no:  2124
Frame no:  2125
Frame no:  2126
Frame no:  2127
Frame no:  2128
Frame no:  2129
Frame no:  2130
Frame no:  2131
Frame no:  2132
Frame no:  2133
Frame no:  2134
Frame no:  2135
Frame no:  2136
Frame no:  2137
Frame no:  2138
Frame no:  2139
Frame no:  2140
Frame no:  2141
Frame no:  2142
Frame no:  2143
Frame no:  2144
Frame no:  2145
Frame no:  2146
Frame no:  2147
Frame no:  2148
Frame no:  2149
Frame no:  2150
Frame no:  2151
Frame no:  2152
Frame no:  2153
Frame no:  2154
Frame no:  2155
Frame no:  2156
Frame no:  2157
Frame no:  2158
Frame no:  2159
Frame no:  2160
Frame no:  2161
Frame no:  2162
Frame no:  2163
Frame no:  2164
Frame no:  2165
Frame no:  2166
Frame no:  2167
Frame no:  2168
Frame no:  2169
Frame no:  2170
Frame no:  2171
Frame no:  2172
Frame no:  2173
Frame no:  2174
Frame no:  2175
Frame no:  2176
Frame no:  2177
Frame no:  2178
Frame no:  2179
Frame no:  2180
Frame no:  2181
Frame no:  2182
Frame no

Frame no:  2634
Frame no:  2635
Frame no:  2636
Frame no:  2637
Frame no:  2638
Frame no:  2639
Frame no:  2640
Frame no:  2641
Frame no:  2642
Frame no:  2643
Frame no:  2644
Frame no:  2645
Frame no:  2646
Frame no:  2647
Frame no:  2648
Frame no:  2649
Frame no:  2650
Frame no:  2651
Frame no:  2652
Frame no:  2653
Frame no:  2654
Frame no:  2655
Frame no:  2656
Frame no:  2657
Frame no:  2658
Frame no:  2659
Frame no:  2660
Frame no:  2661
Frame no:  2662
Frame no:  2663
Frame no:  2664
Frame no:  2665
Frame no:  2666
Frame no:  2667
Frame no:  2668
Frame no:  2669
Frame no:  2670
Frame no:  2671
Frame no:  2672
Frame no:  2673
Frame no:  2674
Frame no:  2675
Frame no:  2676
Frame no:  2677
Frame no:  2678
Frame no:  2679
Frame no:  2680
Frame no:  2681
Frame no:  2682
Frame no:  2683
Frame no:  2684
Frame no:  2685
Frame no:  2686
Frame no:  2687
Frame no:  2688
Frame no:  2689
Frame no:  2690
Frame no:  2691
Frame no:  2692
Frame no:  2693
Frame no:  2694
Frame no:  2695
Frame no

Frame no:  3147
Frame no:  3148
Frame no:  3149
Frame no:  3150
Frame no:  3151
Frame no:  3152
Frame no:  3153
Frame no:  3154
Frame no:  3155
Frame no:  3156
Frame no:  3157
Frame no:  3158
Frame no:  3159
Frame no:  3160
Frame no:  3161
Frame no:  3162
Frame no:  3163
Frame no:  3164
Frame no:  3165
Frame no:  3166
Frame no:  3167
Frame no:  3168
Frame no:  3169
Frame no:  3170
Frame no:  3171
Frame no:  3172
Frame no:  3173
Frame no:  3174
Frame no:  3175
Frame no:  3176
Frame no:  3177
Frame no:  3178
Frame no:  3179
Frame no:  3180
Frame no:  3181
Frame no:  3182
Frame no:  3183
Frame no:  3184
Frame no:  3185
Frame no:  3186
Frame no:  3187
Frame no:  3188
Frame no:  3189
Frame no:  3190
Frame no:  3191
Frame no:  3192
Frame no:  3193
Frame no:  3194
Frame no:  3195
Frame no:  3196
Frame no:  3197
Frame no:  3198
Frame no:  3199
Frame no:  3200
Frame no:  3201
Frame no:  3202
Frame no:  3203
Frame no:  3204
Frame no:  3205
Frame no:  3206
Frame no:  3207
Frame no:  3208
Frame no

Frame no:  3660
Frame no:  3661
Frame no:  3662
Frame no:  3663
Frame no:  3664
Frame no:  3665
Frame no:  3666
Frame no:  3667
Frame no:  3668
Frame no:  3669
Frame no:  3670
Frame no:  3671
Frame no:  3672
Frame no:  3673
Frame no:  3674
Frame no:  3675
Frame no:  3676
Frame no:  3677
Frame no:  3678
Frame no:  3679
Frame no:  3680
Frame no:  3681
Frame no:  3682
Frame no:  3683
Frame no:  3684
Frame no:  3685
Frame no:  3686
Frame no:  3687
Frame no:  3688
Frame no:  3689
Frame no:  3690
Frame no:  3691
Frame no:  3692
Frame no:  3693
Frame no:  3694
Frame no:  3695
Frame no:  3696
Frame no:  3697
Frame no:  3698
Frame no:  3699
Frame no:  3700
Frame no:  3701
Frame no:  3702
Frame no:  3703
Frame no:  3704
Frame no:  3705
Frame no:  3706
Frame no:  3707
Frame no:  3708
Frame no:  3709
Frame no:  3710
Frame no:  3711
Frame no:  3712
Frame no:  3713
Frame no:  3714
Frame no:  3715
Frame no:  3716
Frame no:  3717
Frame no:  3718
Frame no:  3719
Frame no:  3720
Frame no:  3721
Frame no

Frame no:  4083
Frame no:  4084
Frame no:  4085
Frame no:  4086
Frame no:  4087
Frame no:  4088
Frame no:  4089
Frame no:  4090
Frame no:  4091
Frame no:  4092
Frame no:  4093
Frame no:  4094
Frame no:  4095
Frame no:  4096
Frame no:  4097
Frame no:  4098
Frame no:  4099
Frame no:  4100
Frame no:  4101
Frame no:  4102
Frame no:  4103
Frame no:  4104
Frame no:  4105
Frame no:  4106
Frame no:  4107
Frame no:  4108
Frame no:  4109
Frame no:  4110
Frame no:  4111
Frame no:  4112
Frame no:  4113
Frame no:  4114
Frame no:  4115
Frame no:  4116
Frame no:  4117
Frame no:  4118
Frame no:  4119
Frame no:  4120
Frame no:  4121
Frame no:  4122
Frame no:  4123
Frame no:  4124
Frame no:  4125
Frame no:  4126
Frame no:  4127
Frame no:  4128
Frame no:  4129
Frame no:  4130
Frame no:  4131
Frame no:  4132
Frame no:  4133
Frame no:  4134
Frame no:  4135
Frame no:  4136
Frame no:  4137
Frame no:  4138
Frame no:  4139
Frame no:  4140
Frame no:  4141
Frame no:  4142
Frame no:  4143
Frame no:  4144
Frame no

Frame no:  4596
Frame no:  4597
Frame no:  4598
Frame no:  4599
Frame no:  4600
Frame no:  4601
Frame no:  4602
Frame no:  4603
Frame no:  4604
Frame no:  4605
Frame no:  4606
Frame no:  4607
Frame no:  4608
Frame no:  4609
Frame no:  4610
Frame no:  4611
Frame no:  4612
Frame no:  4613
Frame no:  4614
Frame no:  4615
Frame no:  4616
Frame no:  4617
Frame no:  4618
Frame no:  4619
Frame no:  4620
Frame no:  4621
Frame no:  4622
Frame no:  4623
Frame no:  4624
Frame no:  4625
Frame no:  4626
Frame no:  4627
Frame no:  4628
Frame no:  4629
Frame no:  4630
Frame no:  4631
Frame no:  4632
Frame no:  4633
Frame no:  4634
Frame no:  4635
Frame no:  4636
Frame no:  4637
Frame no:  4638
Frame no:  4639
Frame no:  4640
Frame no:  4641
Frame no:  4642
Frame no:  4643
Frame no:  4644
Frame no:  4645
Frame no:  4646
Frame no:  4647
Frame no:  4648
Frame no:  4649
Frame no:  4650
Frame no:  4651
Frame no:  4652
Frame no:  4653
Frame no:  4654
Frame no:  4655
Frame no:  4656
Frame no:  4657
Frame no